In [1]:
import pandas as pd
import numpy as np
from pymongo import *


db = client.xCluster
pd.set_option('display.max_columns', None)

In [2]:
class search_pre_connect_to_database():
    def __init__(self, username):
        self.asst = username

    def list_material_karte(self): #output_assistent_relevant_material_karte_collection
        Assistent = self.asst
        collection_material_karte = db["material_karte"]
        condition = {'Assistent': Assistent}
        dt_alloutput = collection_material_karte.find(condition)
        dt_alloutput_list = list()
        for x in dt_alloutput:
            dt_alloutput_list.append(x)
        return dt_alloutput_list

    def list_soll_chemische_zusammensetzung(self):
        collection_material_karte = db["soll_chemische_zusammensetzung"]
        dt_alloutput = collection_material_karte.find({})
        dt_alloutput_list = list()
        for x in dt_alloutput:
            dt_alloutput_list.append(x)
        return dt_alloutput_list

In [3]:
class search_export_user_related_dataframe():
    def __init__(self, username):
        self.material_karte = search_pre_connect_to_database(username).list_material_karte()
        self.soll_chemische_zusammensetzung = search_pre_connect_to_database(username).list_soll_chemische_zusammensetzung()

    def property_rename(self, property):
        if property == 'Tensile Strength':
            name = 'Zugfestigkeit_Mpa'
            return name
        elif property == 'E-modul':
            name = 'E-modul_Gpa'
            return name
        elif property == 'Yield Stength':
            name = 'Streckgrenze_Mpa'
            return name
        elif property == 'Fracture Strain':
            name = 'Bruchdehnung_%'
            return name
        else:
            return property

    def dataframe_mechanical_property(self): # Temperature in String, together with Material_ID, vor_Material_ID and Treatment_ID
        dt_alloutput_list = self.material_karte # import from function list_material_karte()
        df_test = pd.DataFrame(dt_alloutput_list)
        try:
            df_under_physikalische_Eigenschaft = df_test['physikalische_Eigenschaft']
            df_2 = df_under_physikalische_Eigenschaft.apply(pd.Series)
            denest_key_list = ['mechanische_Eigenschaft', 'Temperatur_°C', 'RT', 'Zugwinkel_°', '90', 'Zuggeschwingkeit_1/s', '0,0025', 'Versuch_id']
            for i in denest_key_list:
                df_2 = df_2[i].apply(pd.Series)
            #name = self.property_rename(property)
            #df_final = pd.DataFrame(columns=["Material_ID", "vor_material_id", 'Treatment_id', name, 'Zugfestigkeit_Mpa'])
            name_list = ['Zugfestigkeit_Mpa', 'E-modul_Gpa', 'Streckgrenze_Mpa', 'Bruchdehnung_%']
            df_final = pd.DataFrame(columns=["Material_ID", "vor_material_id", 'Treatment_id', 'Zugfestigkeit_Mpa', 'E-modul_Gpa', 'Streckgrenze_Mpa', 'Bruchdehnung_%'])
            for name in name_list:
                df_temp_4_mean = pd.DataFrame(columns=[0]).apply(pd.Series)
                for index, row in df_2.iteritems():
                    a = df_2[index].apply(pd.Series)
                    b = a[name].apply(pd.Series)
                    df_temp_4_mean = pd.concat([df_temp_4_mean, b], axis=1)
                mean = df_temp_4_mean.mean(1)
                df_final[name] = mean
            final_dataframe_column = ["Material_ID", "vor_material_id", 'Treatment_id']
            for i in final_dataframe_column:
                df_final[i] = df_test[i]
            df_final = df_final.reset_index(drop=True)
            df_final = df_final.replace(np.nan, "No Data")
            return df_final
        except KeyError:
            df_final = pd.DataFrame(
                columns=["Material_ID", "vor_material_id", 'Treatment_id', 'Zugfestigkeit_Mpa', 'E-modul_Gpa',
                         'Streckgrenze_Mpa', 'Bruchdehnung_%'])
            final_dataframe_column = ["Material_ID", "vor_material_id", 'Treatment_id']
            for i in final_dataframe_column:
                df_final[i] = df_test[i]
            df_final = df_final.reset_index(drop=True)
            df_final = df_final.replace(np.nan, "No Data")
            return df_final

    def dataframe_chemical_composition(self):
        dt_material_karte_output_list = self.material_karte  # import from function list_material_karte()
        df_material_karte = pd.DataFrame(dt_material_karte_output_list)
        drop_list_a = ['vor_material_id', 'Treatment_id', 'physikalische_Eigenschaft', 'Maschinendaten', 'Assistent', '_id', 'Datum', 'Projekt', 'Hersteller', 'Eigenname_Material']
        for i in drop_list_a:
            try:
                df_material_karte = df_material_karte.drop([i], axis=1)
            except KeyError:
                continue
        dt_alloutput_list = self.soll_chemische_zusammensetzung
        df_soll_chemische_zusammensetzung = pd.DataFrame(dt_alloutput_list)
        df_soll_chemische_zusammensetzung = df_soll_chemische_zusammensetzung.drop(['Material_ID', '_id'], axis=1)
        df_merge = pd.merge(df_material_karte, df_soll_chemische_zusammensetzung, on="Werkstoff_id")
        df_merge = df_merge.replace(np.nan, 'None')
        drop_list_b = ['Soll_Chemische_Zusammensetzung_ID', 'Fe']
        for i in drop_list_b:
            try:
                df_merge = df_merge.drop([i], axis=1)
            except KeyError:
                continue

        try:
            df_final = df_merge.loc[df_merge['chemische_Eigenschaft'] != 'None']
            df_under_chemische_Eigenschaft = df_final['chemische_Eigenschaft']
            df_temp = df_under_chemische_Eigenschaft.apply(pd.Series)
            denest_key_list = ['chemische_Zusammensetzung', 'Sample_id']
            for i in denest_key_list:
                df_temp = df_temp[i].apply(pd.Series)
            for index, row in df_temp.iteritems():
                a = df_temp[index].apply(pd.Series)
            df_analyzed_chemical_composition = pd.DataFrame(columns=["Material_ID", "Werkstoff_id"])
            final_dataframe_column = ["Material_ID", "Werkstoff_id"]
            for i in final_dataframe_column:
                df_analyzed_chemical_composition[i] = df_final[i]
            df_analyzed_chemical_composition = pd.concat([df_analyzed_chemical_composition, a], axis=1)
            df_set_chemical_composition = df_merge.loc[df_merge['chemische_Eigenschaft'] == 'None']
            df_set_chemical_composition = df_set_chemical_composition.drop(['chemische_Eigenschaft'], axis=1)
            frames = [df_set_chemical_composition, df_analyzed_chemical_composition]
            df_final_new = pd.concat(frames)
            df_final_new = df_final_new.replace(np.nan, "No Data")
            df_final_new = df_final_new.replace('None', "No Data")
            return df_final_new
        except KeyError:
            try:
                df_set_chemical_composition = df_merge.drop(['Soll_Chemische_Zusammensetzung_ID'], axis=1)
                df_set_chemical_composition = df_set_chemical_composition.replace('None', 'Not Analyzed')
                return df_set_chemical_composition
            except KeyError:
                df_set_chemical_composition = df_merge.replace('None', "No Data")
                return df_set_chemical_composition

    def output(self):
        df_chemical_composition = self.dataframe_chemical_composition()
        df_mechanical_property =self.dataframe_mechanical_property()
        df_result = pd.merge(df_mechanical_property, df_chemical_composition, on='Material_ID')
        return df_result

    def output_queriable_key(self):
        outputlist = list(self.output())
        return outputlist

In [4]:
class outputone_query():
    def __init__(self, username):
        self.dataframe_user = search_export_user_related_dataframe(username).output()

    def one_query(self, query_condition, query_value, query_value_between_upper, evaluate_condition):
        dataframe_all = self.dataframe_user
        try:
            if evaluate_condition == "greater than":
                dataframe_one = dataframe_all.loc[dataframe_all[query_condition] > query_value]
                return dataframe_one
            elif evaluate_condition == "greater than or equal to":
                dataframe_one = dataframe_all.loc[dataframe_all[query_condition] >= query_value]
                return dataframe_one
            elif evaluate_condition == "equal to":
                dataframe_one = dataframe_all.loc[dataframe_all[query_condition] == query_value]
                return dataframe_one
            elif evaluate_condition == "less than or equal to":
                dataframe_one = dataframe_all.loc[dataframe_all[query_condition] <= query_value]
                return dataframe_one
            elif evaluate_condition == "less than":
                dataframe_one = dataframe_all.loc[dataframe_all[query_condition] < query_value]
                return dataframe_one
            elif evaluate_condition == "between":
                if query_value_between_upper < query_value:
                    return 'Query Value error, please input new value.'
                elif query_value_between_upper == query_value:
                    dataframe_one = dataframe_all.loc[dataframe_all[query_condition] == query_value]
                    return dataframe_one
                elif query_value_between_upper > query_value:
                    dataframe_one = dataframe_all.loc[dataframe_all[query_condition] <= query_value_between_upper]
                    dataframe_one = dataframe_one.loc[dataframe_one[query_condition] >= query_value]
                    return dataframe_one
        except KeyError:
            return 'No Query condition'

    def and_or(self, logical_connection):
        output = 0
        try:
            if logical_connection == 'and':
                output = 1
                return output
            elif logical_connection == 'or':
                output = 2
                return output
            elif logical_connection == 'none':
                return output
        except Exception:
            return output

In [22]:
class finaloutput():
    def __init__(self, input_list):
        self.inputlist = input_list

    def extract_output(self):
        list_input = self.inputlist
        list_evaluate_temp = list()
        for i in range(1,len(list_input)):
            list_elem = []
            dataframe_elem = outputone_query(list_input[0]).one_query(list_input[i][0], list_input[i][1], list_input[i][2], list_input[i][3])
            logical_connection_elem = outputone_query(list_input[0]).and_or(list_input[i][4])
            list_elem.append(dataframe_elem)
            list_elem.append(logical_connection_elem)
            list_evaluate_temp.append(list_elem)
            
        #return list_evaluate_temp
# first step
        for i in range(0, len(list_evaluate_temp)):
            if isinstance(list_evaluate_temp[i][0], pd.DataFrame):
                pass
            elif isinstance(list_evaluate_temp[i][0], str):
                return list_evaluate_temp[i][0]
        if list_evaluate_temp[0][1] == 0:
            return list_evaluate_temp[0][0]
        elif list_evaluate_temp[0][1] == 1:
            first = pd.merge(list_evaluate_temp[0][0], list_evaluate_temp[1][0], how='inner')
            if len(list_evaluate_temp) > 3:
                for i in range(2, len(list_evaluate_temp)):
                    j = i-1
                    if list_evaluate_temp[j][1] == 1:
                        first = pd.merge(first, list_evaluate_temp[i][0], how="inner")
                    elif list_evaluate_temp[j][1] == 2:
                        first = pd.merge(first, list_evaluate_temp[i][0], how="outer")
                return first
            else:
                return first
        elif list_evaluate_temp[0][1] == 2:
            first = pd.merge(list_evaluate_temp[0][0], list_evaluate_temp[1][0], how='outer')
            if len(list_evaluate_temp) > 3:
                for i in range(2, len(list_evaluate_temp)):
                    j = i-1
                    if list_evaluate_temp[j][1] == 1:
                        first = pd.merge(first, list_evaluate_temp[i][0], how="inner")
                    elif list_evaluate_temp[j][1] == 2:
                        first = pd.merge(first, list_evaluate_temp[i][0], how="outer")
                return first
            else:
                return first
            
    def user_input_to_get_output(self):
        data = self.extract_output()
        if data.empty:
            print("No entry matching the queries")
        else:
            mat_id_list = list(data.get('Material_ID'))
            print(mat_id_list)
            mat_id_user = input("Enter the Material ID from the above list to check details:")
            print(list(data.keys()))
            mat_id_key = input("Enter the key which you want to see from the above list:")
            if mat_id_user in mat_id_list:
                entry = data.loc[data['Material_ID'] == mat_id_user].get([mat_id_key])
                print(entry)
                user_input = input("Do you want to see the complete details (Yes/No):")
                if user_input == 'Yes':
                    entry_new = data.loc[data['Material_ID'] == mat_id_user]
                    print(entry_new)
                elif user_input == 'No':
                    print('Exiting')
                else:
                    pass
            else:
                print("Entered Material ID not in list")


In [24]:
Assistent_1 = 'Charline Blankart'
Assistent_2 = "Sebastian Wesselmecking"
query_key = "Zugfestigkeit_Mpa"
query_value = 1000
query_value_between_upper = 0
evaluate_condition_0 = "between"
evaluate_condition = "greater than"
logical_connection = "and"
query_key_2 = "E-modul_Gpa"
query_value_2 = 210
query_value_between_upper_2 = 0
evaluate_condition_2 = "greater than or equal to"
logical_connection_2 = "and"
query_key_3 = "Bruchdehnung_%"
query_value_3 = 10
query_value_between_upper_3 = 0
logical_connection_3 = "and"
query_key_4 = "Streckgrenze_Mpa"
query_value_4 = 1500
query_value_between_upper_4 = 0
logical_connection_4 = "none"

list_input = [Assistent_2, [query_key, query_value, query_value_between_upper, evaluate_condition, logical_connection], [query_key_2, query_value_2, query_value_between_upper_2,evaluate_condition_2, logical_connection_2],
              [query_key_3, query_value_3,query_value_between_upper_3, evaluate_condition, logical_connection_3],[query_key_4, query_value_4,query_value_between_upper_4, evaluate_condition_2, logical_connection_4]]

list_input_2 = [Assistent_2,[query_key, query_value, query_value_between_upper, evaluate_condition, logical_connection]]

In [25]:
f = finaloutput(list_input)
g = f.user_input_to_get_output()

['SW10036']
Enter the Material ID from the above list to check details:SW10036
['Material_ID', 'vor_material_id', 'Treatment_id', 'Zugfestigkeit_Mpa', 'E-modul_Gpa', 'Streckgrenze_Mpa', 'Bruchdehnung_%', 'Werkstoff_id', 'Al', 'C', 'Mn', 'Si', 'Cr', 'Legierungsbezeichnung', 'P', 'S', 'Ni', 'Mo', 'Cu', 'Co', 'Nb', 'V', 'Sn', 'N', 'As', 'B', 'Bi', 'Ca', 'Ce', 'La', 'Mg', 'Pb', 'Sb', 'Ti', 'W', 'Zn', 'Zr']
Enter the key which you want to see from the above list:Al
     Al
0  1.21
Do you want to see the complete details (Yes/No):Yes
  Material_ID vor_material_id Treatment_id  Zugfestigkeit_Mpa  E-modul_Gpa  \
0     SW10036         SW10004   SW10036_HT             1690.0        225.0   

   Streckgrenze_Mpa  Bruchdehnung_% Werkstoff_id    Al     C    Mn    Si  \
0            1506.5            13.0      WK10001  1.21  0.39  18.8  0.28   

     Cr Legierungsbezeichnung        P        S       Ni       Mo       Cu  \
0  2.42               No Data  No Data  No Data  No Data  No Data  No Data   
